In [1]:
from dotenv import load_dotenv
import os

# .env 파일에서 API 키 로드
load_dotenv()

True

In [2]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# 1단계: 장르별 영화 추천 프롬프트
movie_recommendation_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 영화 전문가입니다. 사용자가 요청한 장르에 맞는 영화 1편을 추천하고 간단한 설명을 제공해주세요."),
    ("user", "{genre} 장르의 한국영화 1편을 추천해주세요. 영화 제목과 왜 이 영화를 추천하는지 이유도 함께 알려주세요.")
])

In [3]:
plot_summary_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 영화 줄거리 요약 전문가입니다. 영화의 핵심 내용을 3줄로 간결하게 요약해주세요."),
    ("user", "다음 영화 추천 내용을 바탕으로 해당 영화의 등장인물과 줄거리를 정확히 10줄로 요약해주세요: {movie}")
])

In [7]:
# LLM 모델 초기화
# llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7)
llm = ChatOpenAI(
    api_key= os.getenv("GROQ_API_KEY"),
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)

# 1단계 체인: 영화 추천
recommendation_chain = movie_recommendation_prompt | llm | StrOutputParser()
# Step 1: 사용자가 입력한 장르에 따라 영화 추천
movie = recommendation_chain.invoke({"genre": "Drama"})  # 영화 제목 얻기
print(" 추천된 영화:", movie)  # movie 값 출력

# 2단계 체인: 줄거리 요약 (1단계 결과를 입력으로 받음)
plot_chain = (
    {"movie": recommendation_chain}  # 1단계 체인의 결과를 movie로 전달
    | plot_summary_prompt
    | llm
    | StrOutputParser()
)

 추천된 영화: ### 드라마 영화 추천

*   **영화 제목:** **"올드보이" (2003)**
*   **추천 이유:** '올드보이'는 미스터리한 스토리와 강렬한 연출로 유명한 작품입니다. 주인공 오대수는 15년간 감금된 후, 자신의 과거를 복수하려는 여정에 나섭니다. 이 영화는 한국 드라마의 정수를 보여주며, 복잡한 플롯과 감동적인 결말이 특징입니다.

'올드보이'는 한국 영화의 대표작 중 하나로, 많은 영화 팬들이 추천하는 드라마 장르의 영화입니다.


In [8]:
# 테스트 실행
try:
    genre = "Drama"
    
    print("영화 추천 시스템")
    print("==================")
    print(f"\n입력 장르: {genre}")
    
    # 1단계 실행 및 결과 출력
    movie_recommendation = recommendation_chain.invoke({"genre": genre})
    print("\n1단계 - 영화 추천 결과:")
    print(movie_recommendation)
    
    print("\n---\n")
    
    # 2단계 실행 및 결과 출력
    plot_summary = plot_chain.invoke({"genre": genre})
    print("2단계 - 영화 줄거리 요약:")
    print(plot_summary)
    
except Exception as e:
    print(f"오류 발생: {e}")

영화 추천 시스템

입력 장르: Drama

1단계 - 영화 추천 결과:
1.  영화: '올드보이' (2003)
2.  이유: '올드보이'는 드라마 장르의 한국 영화로, 박찬욱 감독의 작품입니다. 이 영화는 복수에 대한 깊은 고찰을 담고 있으며, 심리적인 긴장감과 강렬한 연기로 많은 사랑을 받았습니다. 또한, 이 영화는 한국 영화의 우수성을 세계에 알린 작품 중 하나입니다.

---

2단계 - 영화 줄거리 요약:
'올드보이'는 한국의 상징적인 영화 중 하나로, 주인공 오대수(최민식)가 15년간 납치되어 갇혀 있는 동안 겪는 경험과 그 이후의 이야기를 중심으로 합니다. 오대수는 납치된 후, 알 수 없는 곳에서 15년간 갇혀 지내게 됩니다. 어느 날 갑자기 석방된 오대수는 자신의 납치와 감금의 이유를 찾기 시작합니다. 

이를 통해 그는 과거의 비밀과 복수의 욕망에 빠져들게 됩니다. 그는 자신의 과거와 납치의 배후에 대해 조사하며, 진실을 밝혀내기 위해 노력합니다. 

오대수의 여정은 인간의 본성과 복수, 용서의 테마를 심오하게 탐구하며, 강렬한 연기와 뛰어난 연출로 많은 관객에게 깊은 인상을 남깁니다. 영화는 칸 영화제에서 '심사위원 대상'을 수상하는 등 국제적으로도 높은 평가를 받았습니다.


In [9]:
# 다른 장르로 추가 테스트
genre2 = "로맨스"

movie_recommendation2 = recommendation_chain.invoke({"genre": genre2})
plot_summary2 = plot_chain.invoke({"genre": genre2})

print(f"다른 장르로 테스트 - {genre2}:")
print("===============================")
print("\n영화 추천:")
print(movie_recommendation2)
print("\n줄거리 요약:")
print(plot_summary2)

다른 장르로 테스트 - 로맨스:

영화 추천:
로맨스 장르의 한국영화 1편으로 '건축학개론' (2012)을 추천합니다. 이 영화는 로맨스 장르의 대표작 중 하나이며, 한국영화 최초로 천만 관객을 돌파한 작품입니다.

'건축학개론'은 두 남녀의 첫사랑과 재회를 다루는 영화입니다. 대학생 시절, 건축학과 학생인 '서준' (정재영)과 '선희' (한가인)은 서로에게 첫사랑의 감정을 느낍니다. 하지만, 서준은 선희에게 고백하지 못하고, 선희는 서준의 감정을 모른 채 다른 사람과 결혼합니다. 

20년 후, 서준과 선희는 재회하게 되고, 서로에 대한 감정이 여전히 남아 있음을 깨닫습니다. 

이 영화는 첫사랑의 아픔과 재회의 설렘을 섬세하게 표현하며, 많은 관객들에게 사랑을 받았습니다. 건축학개론은 로맨틱한 분위기와 감동적인 이야기로 가득 차 있어, 로맨스 장르를 좋아하는 분들에게 꼭 추천하는 영화입니다.

줄거리 요약:
Here is a summary of the movie "올드미스" (2008) in 10 lines:

* 영화 제목: 올드미스 (2008)
* 주인공: 30대 여성 
* 줄거리: 주인공은 성공한 사업가와 사랑에 빠지게 됩니다.
* 사업가는 매우 성공한 사람으로, 외모와 재능을 모두 갖춘 인물입니다.
* 두 사람은 서로 다른 세계에 속해 있지만, 서로에게 끌리게 됩니다.
* 주인공은 사업가와의 사랑을 통해 삶의 의미를 찾게 됩니다.
* 영화는 두 사람의 사랑과 성장을 섬세하게 표현합니다.
* 영화는 감동적인 연기와 감정 표현으로 많은 사랑을 받았습니다.
* 영화의 결말은 감동적이며, 깊은 감동을 전달합니다.
* 올드미스는 한국 로맨스 영화의 대표적인 작품 중 하나입니다.


In [10]:
# 체인 연결 방식 설명
print("체인 연결 방식 설명:")
print("====================")
print()
print("1단계 체인:")
print("movie_recommendation_prompt | llm | StrOutputParser()")
print()
print("2단계 체인:")
print('{"movie": recommendation_chain} | plot_summary_prompt | llm | StrOutputParser()')
print()
print("핵심 포인트:")
print("- 1단계 체인의 출력이 2단계 체인의 입력으로 자동 전달")
print("- movie 키를 통해 데이터 흐름 제어")
print("- LCEL을 통해 직관적인 파이프라인 구성")

체인 연결 방식 설명:

1단계 체인:
movie_recommendation_prompt | llm | StrOutputParser()

2단계 체인:
{"movie": recommendation_chain} | plot_summary_prompt | llm | StrOutputParser()

핵심 포인트:
- 1단계 체인의 출력이 2단계 체인의 입력으로 자동 전달
- movie 키를 통해 데이터 흐름 제어
- LCEL을 통해 직관적인 파이프라인 구성
